In [1]:
import json
from bs4 import Tag
from tqdm import tqdm
import sertha_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:
file_name_code = "sertha"

In [3]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

# def read_json(path, file_name):
#     try:
#         with open(path+file_name, 'r', encoding='utf-8') as openfile:
#             Loaded_file = json.load(openfile)
#             print(f"Successfully loaded: {file_name}")
#         return Loaded_file
#     except Exception as e:
#         print(f"Error loading {file_name}: {str(e)}")
#         return None

def scrape_article(url, page_key_code):
    try:
        article_content = sertha_utils.scrape_sertha_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_lenght+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_{file_name_code}_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"{file_name_code}_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [4]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in sertha_ALL_link_༧གོང་ས་མཆོག.json: 3
page key name: Page ༧གོང་ས་མཆོག 3
Page key code: Page ༧གོང་ས་མཆོག 
Total page in sertha_ALL_link_རྩོམ་རིག.json: 5
page key name: Page རྩོམ་རིག 5
Page key code: Page རྩོམ་རིག 
Total page in sertha_ALL_link_ལོ་རྒྱུས།.json: 2
page key name: Page ལོ་རྒྱུས། 2
Page key code: Page ལོ་རྒྱུས། 
Total page in sertha_ALL_link_སློབ་གསོ།.json: 1
page key name: Page སློབ་གསོ། 1
Page key code: Page སློབ་གསོ། 
Total page in sertha_ALL_link_གསར་རྩོམ།.json: 1
page key name: Page གསར་རྩོམ། 1
Page key code: Page གསར་རྩོམ། 
Total page in sertha_ALL_link_གསར་འགྱུར།.json: 2
page key name: Page གསར་འགྱུར། 2
Page key code: Page གསར་འགྱུར། 
Total page in sertha_ALL_link_མི་སྣ་ངོ་སྤྲོད།.json: 1
page key name: Page མི་སྣ་ངོ་སྤྲོད། 1
Page key code: Page མི་སྣ་ངོ་སྤྲོད། 
Total page in sertha_ALL_link_སྲོལ་རྒྱུན།.json: 1
page key name: Page སྲོལ་རྒྱུན། 1
Page key code: Page སྲོལ་རྒྱུན། 


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]


  0%|          | 0/8 [00:00<?, ?it/s]



  0%|          | 0/20 [00:00<?, ?it/s]




  0%|          | 0/8 [00:00<?, ?it/s]





  0%|          | 0/29 [00:00<?, ?it/s]






  0%|          | 0/18 [00:00<?, ?it/s]







  0%|          | 0/60 [00:00<?, ?it/s]





  3%|▎         | 1/29 [00:06<03:14,  6.94s/it]



  5%|▌         | 1/20 [00:07<02:19,  7.32s/it]







  2%|▏         | 1/60 [00:07<07:12,  7.34s/it]



 10%|█         | 2/20 [00:07<00:55,  3.10s/it]


 14%|█▍        | 1/7 [00:08<00:48,  8.03s/it]







  5%|▌         | 3/60 [00:08<02:03,  2.16s/it]


 29%|██▊       | 2/7 [00:08<00:16,  3.40s/it]





  7%|▋         | 2/29 [00:08<01:38,  3.65s/it]



 15%|█▌        | 3/20 [00:08<00:36,  2.13s/it]


 43%|████▎     | 3/7 [00:08<00:08,  2.02s/it]

 33%|███▎      | 1/3 [00:08<00:17,  8.59s/it]



 20%|██        | 4/20 [00:08<00:21,  1.35s/it]





 10%|█         | 3/29 [00:08<00:56,  2.18s/it]



 25%|█

Total Failure in the སྲོལ་རྒྱུན། article: 0
sertha_ALL_content_སྲོལ་རྒྱུན།.json
Successfully saved: sertha_ALL_content_སྲོལ་རྒྱུན།.json









 67%|██████▋   | 6/9 [00:12<00:03,  1.02s/it]]



 45%|████▌     | 9/20 [00:12<00:11,  1.07s/it]






 39%|███▉      | 7/18 [00:13<00:09,  1.11it/s]





 24%|██▍       | 7/29 [00:13<00:24,  1.13s/it]







 10%|█         | 6/60 [00:13<01:36,  1.78s/it]


 78%|███████▊  | 7/9 [00:13<00:01,  1.11it/s]




 62%|██████▎   | 5/8 [00:13<00:05,  1.98s/it]



 50%|█████     | 10/20 [00:13<00:09,  1.02it/s]






 44%|████▍     | 8/18 [00:13<00:08,  1.21it/s]





 28%|██▊       | 8/29 [00:13<00:20,  1.03it/s]







 89%|████████▉ | 8/9 [00:13<00:00,  1.36it/s]]




100%|██████████| 9/9 [00:13<00:00,  1.49s/it]


Total Failure in the སློབ་གསོ། article: 0
sertha_ALL_content_སློབ་གསོ།.json
Successfully saved: sertha_ALL_content_སློབ་གསོ།.json


...

Total Failure in the གསར་འགྱུར། article: 0
sertha_ALL_content_གསར་འགྱུར།.json
Successfully saved: sertha_ALL_content_གསར་འགྱུར།.json








 83%|████████▎ | 24/29 [00:23<00:01,  3.23it/s]







 42%|████▏     | 25/60 [00:23<00:11,  3.15it/s]





 90%|████████▉ | 26/29 [00:27<00:02,  1.06it/s]







 43%|████▎     | 26/60 [00:27<00:41,  1.21s/it]







 45%|████▌     | 27/60 [00:28<00:34,  1.05s/it]





 93%|█████████▎| 27/29 [00:29<00:02,  1.10s/it]







 47%|████▋     | 28/60 [00:29<00:35,  1.12s/it]







 48%|████▊     | 29/60 [00:29<00:27,  1.14it/s]







 52%|█████▏    | 31/60 [00:29<00:16,  1.77it/s]







 53%|█████▎    | 32/60 [00:30<00:16,  1.69it/s]





100%|██████████| 29/29 [00:30<00:00,  1.06s/it]








 12%|█▎        | 1/8 [00:30<03:36, 30.94s/it]

Total Failure in the ༧གོང་ས་མཆོག article: 0
sertha_ALL_content_༧གོང་ས་མཆོག.json
Successfully saved: sertha_ALL_content_༧གོང་ས་མཆོག.json










 57%|█████▋    | 34/60 [00:31<00:12,  2.07it/s]







 58%|█████▊    | 35/60 [00:31<00:10,  2.38it/s]







 60%|██████    | 36/60 [00:32<00:13,  1.82it/s]







 63%|██████▎   | 38/60 [00:32<00:07,  2.92it/s]







 67%|██████▋   | 40/60 [00:32<00:04,  4.19it/s]







 68%|██████▊   | 41/60 [00:38<00:26,  1.37s/it]







 73%|███████▎  | 44/60 [00:38<00:13,  1.20it/s]







 75%|███████▌  | 45/60 [00:39<00:12,  1.25it/s]







 78%|███████▊  | 47/60 [00:39<00:07,  1.78it/s]







 80%|████████  | 48/60 [00:40<00:05,  2.03it/s]







 82%|████████▏ | 49/60 [00:40<00:05,  1.95it/s]







 83%|████████▎ | 50/60 [00:41<00:05,  1.99it/s]







 85%|████████▌ | 51/60 [00:43<00:08,  1.07it/s]







 88%|████████▊ | 53/60 [00:43<00:04,  1.71it/s]







 90%|█████████ | 54/60 [00:43<00:02,  2.07it/s]







 97%|█████████▋| 58/60 [00:44<00:00,  3.48it/s]







 98%|█████████▊| 59/60 [00:47<00:00,  1.16it/s]







100%|██████████| 60/60 [00:49<00:00,  1.21it/s]

100%|███

Total Failure in the རྩོམ་རིག article: 0
sertha_ALL_content_རྩོམ་རིག.json
Successfully saved: sertha_ALL_content_རྩོམ་རིག.json
Processed 8 files
CPU times: user 24.6 s, sys: 657 ms, total: 25.2 s
Wall time: 49.7 s


In [ ]:
# 14:44

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files